#Notebook para la construcción de la dimensión de tiempo

###0.Definir listas de errores

In [0]:
log_error_list = []

###1.Importar librerías

In [0]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
import datetime
import calendar
import numpy
import glob
import os
from os import remove
from datetime import datetime
from delta.tables import *
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
from delta.tables import DeltaTable
from delta.tables import *

###2.Función para crear la dimensión de tiempo

In [0]:
def create_date_table(start='2000-01-01', end='2100-12-31'):
    ...:     df = pd.DataFrame({"Date": pd.date_range(start, end)})
    ...:     df["Day"]=df['Date'].dt.day
    ...:     df["DayName"] = df['Date'].dt.day_name()
    ...:     df["WeekDayNum"]=df["Date"].dt.dayofweek
    ...:     df["Quarter"] = df.Date.dt.quarter
    ...:     df["Year"] = df.Date.dt.year
    ...:     df["Year_half"] = (df.Quarter + 1) // 2
    ...:     df["MonthNum"]=df.Date.dt.month 
    ...:     df["MonthLong"]=df['MonthNum'].apply(lambda x: calendar.month_name[x])
    ...:     df["Month"]=df['MonthNum'].apply(lambda x: calendar.month_abbr[x])
    ...:     df["MonthYear"]=df["Month"].astype('str')+"-"+df["Year"].astype('str').str[-2]+df["Year"].astype('str').str[-1]
    ...:     df["MonthYearLong"]=df["MonthLong"].astype('str')+"-"+df["Year"].astype('str')
    ...:     return df

###3.Creación de la dimensión de Tiempo

In [0]:
if len(log_error_list)>0:
    print("Previous Error")
else:
    try:
      df_Dim_Tiempo=create_date_table()
      #df_Dim_Tiempo

      #Revisión de los primeros 10 registros de la dimensión de tiempo
      #df_Dim_Tiempo.head(10)

      #Revisión de los últimos 10 registros de la dimensión de tiempo
      #df_Dim_Tiempo.tail(10)
      #Revisión de los tipos de dato 
      #df_Dim_Tiempo.dtypes

    except Exception as e:
        print("Failed! Due to: {}".format(str(e)))
        log_error_list.append("Error: {}".format(str(e)))

###4.Convertir a Spark

In [0]:
if len(log_error_list)>0:
    print("Previous Error")
else:
    try:
      #Se convierte el dataframe de pandas a spark
      df_Dim_Tiempo = spark.createDataFrame(df_Dim_Tiempo)
      #display(df_Dim_Tiempo)
      
      #Ver el esquema del dataframe
      #df_Dim_Tiempo.printSchema()

    except Exception as e:
        print("Failed! Due to: {}".format(str(e)))
        log_error_list.append("Error: {}".format(str(e)))

###5. Hacer un Upsert a la tabla de Dim_Tiempo

In [0]:
if len(log_error_list)>0:
    print("Previous Error")
else:
    try:
      presentation_folder_path="/mnt/presentation"
      
      ##Declarar la tabla que se va a sobreescribir y a la que se le va a hacer el Upsert
      deltaTable = DeltaTable.forPath(spark, str(presentation_folder_path)+"/"+"Dim_Tiempo/")

      ###Instrucción para sobreescribir los datos existentes e insertar registros nuevos 
      deltaTable.alias("tgt").merge(
          df_Dim_Tiempo.alias("upd"),
          "tgt.Date= upd.Date") \
        .whenMatchedUpdate(set = { "Date" : "upd.Date", "Day" : "upd.Day", "DayName" : "upd.DayName", "WeekDayNum" : "upd.WeekDayNum", "Quarter" : "upd.Quarter",  "Year" : "upd.Year", "Year_half" : "upd.Year_half", "MonthNum" : "upd.MonthNum", "MonthLong" : "upd.MonthLong", "Month" : "upd.Month", "MonthYear" : "upd.MonthYear", "MonthYearLong" : "upd.MonthYearLong"} ) \
        .whenNotMatchedInsert(values =
          { "Date" : "upd.Date",
            "Day" : "upd.Day",
            "DayName" : "upd.DayName",
            "WeekDayNum" : "upd.WeekDayNum",
            "Quarter" : "upd.Quarter",
            "Year" : "upd.Year",
            "Year_half" : "upd.Year_half",
            "MonthNum" : "upd.MonthNum",
            "MonthLong" : "upd.MonthLong",
            "Month" : "upd.Month",
            "MonthYear" : "upd.MonthYear",
            "MonthYearLong" : "upd.MonthYearLong"
          }
        ) \
        .execute()

    except Exception as e:
        print("Failed! Due to: {}".format(str(e)))
        log_error_list.append("Error: {}".format(str(e)))

###4.Leer y escribir la última versión de la tabla de Dim_Tiempo

####4.1. Leer y escribir última versión

In [0]:
if len(log_error_list)>0:
    print("Previous Error")
else:
    try:
      df_Dim_Tiempo=spark.read.format("delta").load(str(presentation_folder_path)+"/"+"Dim_Tiempo/")
      df_Dim_Tiempo.write.mode("overwrite").format("delta").save(str(presentation_folder_path)+"/"+"Dim_Tiempo/")
      #display(df_Dim_Tiempo)

    except Exception as e:
        print("Failed! Due to: {}".format(str(e)))
        log_error_list.append("Error: {}".format(str(e)))

####4.2. Eliminar las versiones anteriores y dejar solo la última

In [0]:
if len(log_error_list)>0:
    print("Previous Error")
else:
    try:
      ##Desactivar el tiempo de permanencia de las versiones anteriores
      spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled","false")
      
      #Elimnar versiones anteriores 
      deltaTable = DeltaTable.forPath(spark, str(presentation_folder_path)+"/"+"Dim_Tiempo/")
      deltaTable.vacuum(0)

      ##Leer e imprimir la última versión
      #dfVentas=spark.read.format("delta").load(str(processed_folder_path)+"/"+str(cliente)+"/Fact_Ventas/")
      #display(dfVentas)

    except Exception as e:
        print("Failed! Due to: {}".format(str(e)))
        log_error_list.append("Error: {}".format(str(e)))


#5. Estado final del notebook

In [0]:
if len(log_error_list)>0:
    log_status = "Failure"
    log_error_list = list(dict.fromkeys(log_error_list))
    log_error = str(log_error_list)
    rows_read = 0
    rows_written = 0
    print(log_error)
else:
    log_status = "Success"
    log_error = ''
print(log_status)

Success